# Vocabulario y Matching

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ohtar10/icesi-nlp/blob/main/Sesion1/5-vocabulary.ipynb)

## Referencias
* [NLP - Natural Language Processing With Python](https://www.udemy.com/course/nlp-natural-language-processing-with-python)
* [Natural Language Processing in Action](https://www.manning.com/books/natural-language-processing-in-action)

Ahora miremos algunos ejemplos y pruebas con vocabularios en Spacy

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipykernel_1565021/2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/requirements.txt && pip install -r requirements.txt

## Matchers
Podemos pensar en los matchers de forma similar a como usamos las expresiones regulares en programación tradicional, solo que aplicado a documentos.

In [3]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab) # work with the normal vocabulary

In [4]:
# SolarPower
pattern1 = [{'LOWER': 'solarpower'}]
# Solar-power
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True}, {'LOWER': 'power'}]
# Solar power
pattern3 = [{'LOWER': 'solar'}, {'LOWER': 'power'}]

Observemos el ejemplo anterior y los correspondientes patrones que queremos encontrar. Nótese que cada parte puede ser representada como un diccionario. Esto también es una forma de hacer *tagging* de los tokens, más allá de los estándar POS, que pueden ser útiles dependiendo de la tarea en cuestión.

## Otros atributos para los tokens
A parte de los lemmas, hay otra variedad de atributos para los tokens que podemos utilizar para determinar reglas de matching:

<table><tr><th>Attribute</th><th>Description</th></tr>

<tr ><td><span >`ORTH`</span></td><td>The exact verbatim text of a token</td></tr>
<tr ><td><span >`LOWER`</span></td><td>The lowercase form of the token text</td></tr>
<tr ><td><span >`LENGTH`</span></td><td>The length of the token text</td></tr>
<tr ><td><span >`IS_ALPHA`, `IS_ASCII`, `IS_DIGIT`</span></td><td>Token text consists of alphanumeric characters, ASCII characters, digits</td></tr>
<tr ><td><span >`IS_LOWER`, `IS_UPPER`, `IS_TITLE`</span></td><td>Token text is in lowercase, uppercase, titlecase</td></tr>
<tr ><td><span >`IS_PUNCT`, `IS_SPACE`, `IS_STOP`</span></td><td>Token is punctuation, whitespace, stop word</td></tr>
<tr ><td><span >`LIKE_NUM`, `LIKE_URL`, `LIKE_EMAIL`</span></td><td>Token text resembles a number, URL, email</td></tr>
<tr ><td><span >`POS`, `TAG`, `DEP`, `LEMMA`, `SHAPE`</span></td><td>The token's simple and extended part-of-speech tag, dependency label, lemma, shape</td></tr>
<tr ><td><span >`ENT_TYPE`</span></td><td>The token's entity label</td></tr>

</table>

Ahora podemos agregar estos patrones al matcher y aplicarlos a un documento:

In [9]:
patterns = [pattern1, pattern2, pattern3]
matcher.add('SolarPower', patterns)
doc = nlp('The Solar Power industry continues to grow as solarpower increases. \
Solar-power cars are gaining popularity. ')

found_matches = matcher(doc)

In [10]:
found_matches

[(8656102463236116519, 1, 3),
 (8656102463236116519, 8, 9),
 (8656102463236116519, 11, 14)]

La lista obtenida contiene los ID de los tokens y los índices desde donde comienzan y terminan dentro del documento. Podemos imprimir esta información utilizando el vocabulario con los ID de token que hemos obtenido.

In [11]:
def print_matches(found_matches):
    for match_id, start, end in found_matches:
        string_id = nlp.vocab.strings[match_id] # get string representation
        span = doc[start:end] # get a span of this particular match
        print(match_id, string_id, start, end, span.text)

print_matches(found_matches)

8656102463236116519 SolarPower 1 3 Solar Power
8656102463236116519 SolarPower 8 9 solarpower
8656102463236116519 SolarPower 11 14 Solar-power


Aquí podemos observar diferentes marches que apuntan a la misma entrada en el vocabulario (el matcher) y el correspondiente texto al que hace referencia.

Podemos remover los matchers si ya no nos interesa trabajar cone llos.

In [12]:
matcher.remove('SolarPower') # use the name we defined when we added it

Y agregar otro conjunto de marchers

In [13]:
# solarpower SolarPower
pattern1 = [{'LOWER': 'solarpower'}]
# solar.power solar-power etc
pattern2 = [{'LOWER': 'solar'}, {'IS_PUNCT': True, 'OP': '*'}, {'LOWER': 'power'}]

patterns = [pattern1, pattern2]
matcher.add('SolarPower', patterns)

In [14]:
found_matches = matcher(doc)
print_matches(found_matches)

8656102463236116519 SolarPower 1 3 Solar Power
8656102463236116519 SolarPower 8 9 solarpower
8656102463236116519 SolarPower 11 14 Solar-power


Los cuantificadores a continuación pueden ser procesados con el operador correspondiente:
<table><tr><th>OP</th><th>Description</th></tr>

<tr ><td><span >\!</span></td><td>Negate the pattern, by requiring it to match exactly 0 times</td></tr>
<tr ><td><span >?</span></td><td>Make the pattern optional, by allowing it to match 0 or 1 times</td></tr>
<tr ><td><span >\+</span></td><td>Require the pattern to match 1 or more times</td></tr>
<tr ><td><span >\*</span></td><td>Allow the pattern to match zero or more times</td></tr>
</table>


Notemos que hemos obtenido el mismo resultado pero solo con un patrón menos, esto es gracias a la opción `OP` en el segundo patrón la cual hace que, al igual que en regex, haga match con cero o más ocurrencias.

## Matchers de frases
Como otra alternativa para la construcción de patrones, podemos crear unos con frases:

In [15]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

Empecemos a trabajar con un corpus real. Vamos a usar el extracto del articulo de *Reaganomics* de la Wikipedia:

* https://en.wikipedia.org/wiki/Reaganomics

In [19]:
!test '{IN_COLAB}' = 'True' && wget  https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/Sesion1/reaganomics.txt

In [16]:
with open('./reaganomics.txt', encoding='ISO-8859-1') as file:
    doc = nlp(file.read())

Y luego vamos a buscar patrones de frases como:

In [17]:
phrase_list = ['voodoo economics', 'supply-side economics', 'trickle-down economics', 'free-market economics']
phrase_patterns = [nlp(text) for text in phrase_list]
matcher.add('EconMatcher', None, *phrase_patterns)
found_matches = matcher(doc)

In [18]:
print_matches(found_matches)

3680293220734633682 EconMatcher 41 45 supply-side economics
3680293220734633682 EconMatcher 49 53 trickle-down economics
3680293220734633682 EconMatcher 54 56 voodoo economics
3680293220734633682 EconMatcher 61 65 free-market economics
3680293220734633682 EconMatcher 673 677 supply-side economics
3680293220734633682 EconMatcher 2986 2990 trickle-down economics


Hemos encontrado una serie de matches, con sus respectivos índices de donde se encuentran dichas frases.

Esto puede ser útil a la hora de construir vocabularios, recordemos que en NLP, los vocabularios no están limitados exclusivamente a palabras sueltas, muchas veces dos palabras juntas traen un significado diferente que si estuvieran separadas y valdría la pena considerarlas como un token independiente.